In [ ]:
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np

In [ ]:
class person:
    def __init__(self,i,gender, orientation, age, sexual_activity):
        # gender 0 : male, 1 : female
        # orientation 0 : straight, 1 : homosexual
        # time_since_infection equals -1 if not infected
        
        self.identifier = i
        self.sec_id = -1
        self.gender = gender
        self.orientation = orientation
        self.age = age
        self.sexual_activity = sexual_activity
        self.disease_status = 0
        self.time_since_infection = -1
        self.number_of_partners = 0
        self.current_partners = set()
    
    def print_state(self):
        print(self.identifier, 
              self.gender, 
              self.orientation,
              self.age,
              self.sexual_activity,
              self.disease_status,
              self.time_since_infection,
              self.number_of_partners,
              self.current_partners)
    
    def add_partner(self, other_person,s):
        if self.number_of_partners == 0:
            s.number_of_singles -= 1
        if other_person.identifier not in self.current_partners:
            self.number_of_partners += 1
            self.current_partners.add(other_person.identifier)
        
    def remove_partner(self, other_person,s):
        self.number_of_partners -= 1
        self.current_partners.remove(other_person.identifier)
        
        if self.number_of_partners == 0:
            s.number_of_singles += 1
    
    def asymptomatic_infection(self, s):
        if self.disease_status == 0:
            self.disease_status = 2
            self.time_since_infection = 0
            s.number_of_asymptomatic += 1
        
    def symptomatic_infection(self, s):
        if self.disease_status == 0:
            self.disease_status = 1
            self.time_since_infection = 0
            s.number_of_symptomatic += 1
        elif self.disease_status == 2:
            self.disease_status = 1
            s.number_of_symptomatic += 1
            s.number_of_asymptomatic -= 1
    
    def cure(self, s):
        if self.disease_status == 1:
            s.number_of_symptomatic -= 1
        elif self.disease_status == 2:
            s.number_of_asymptomatic -= 1
        self.disease_status = 0
        self.time_since_infection = -1
        
class Partnership:
    def __init__(self,partnership_type,person1,person2):
        """
        type 1 : steady, 2 : casual
        
        """
        self.type = partnership_type
        self.persons = [person1,person2]
    
    def print_state(self):
        print(self.type,
              self.persons[0].identifier,
              self.persons[1].identifier)
        
class system:
    def __init__(self):
        self.persons = []
        self.partnerships = []
        self.straight_males = []
        self.homosexual_males = []
        self.females = []
        self.time = 0
        self.number_of_symptomatic = 0
        self.number_of_asymptomatic = 0
        self.number_of_steady = 0
    
    def set_constants(self,
                      f = 0.2,
                      rho = 0.006,
                      sigma1 = 0.0004,
                      sigma2 = 0.1,
                      transmission_male_female_steady = 0.15,
                      transmission_female_male_steady = 0.0625,
                      transmission_male_female_casual = 0.6,
                      transmission_female_male_casual = 0.25,
                      asymptomatic_men = 0.1,
                      asymptomatic_women = 0.45,
                      incubation_time_men = 5,
                      incubation_time_women = 10,
                      patient_delay_treatment_men = 5,
                      patient_delay_treatment_women = 8,
                      recovery_rate_asymptomatic_men = 0.0074,
                      recovery_rate_symptomatic_men = 0.04,
                      recovery_rate_asymptomatic_women = 0.0044,
                      recovery_rate_symptomatic_women = 0.03,
                      screening_percentage = 0.02,
                      sexual_activity_high = 0.05):
        """
        sets constants

        Definitions
        f: probability of steady partnership
        rho: probability of forming partnership
        sigma1 : probability of steady partnership separation
        sigma2 : probability of casual partnership separation
        """
        self.f = f
        self.rho = rho
        self.sigma1 = sigma1
        self.sigma2 = sigma2

        self.transmission_male_female_steady = transmission_male_female_steady
        self.transmission_female_male_steady = transmission_female_male_steady
        self.transmission_male_female_casual = transmission_male_female_casual
        self.transmission_female_male_casual = transmission_female_male_casual
        self.asymptomatic_men = asymptomatic_men
        self.asymptomatic_women = asymptomatic_women
        self.incubation_time_men = incubation_time_men
        self.incubation_time_women = incubation_time_women
        self.patient_delay_treatment_men = patient_delay_treatment_men
        self.patient_delay_treatment_women = patient_delay_treatment_women
        self.recovery_rate_asymptomatic_men = recovery_rate_asymptomatic_men
        self.recovery_rate_symptomatic_men = recovery_rate_symptomatic_men
        self.recovery_rate_asymptomatic_women = recovery_rate_asymptomatic_women
        self.recovery_rate_symptomatic_women = recovery_rate_symptomatic_women
        self.screening_percentage = screening_percentage
        self.sexual_activity_high = sexual_activity_high

    
    def initialize(self,number_of_persons, gender_ratio, queer_ratio):
        for i in range(number_of_persons):
            gender =  0 if np.random.random() > gender_ratio else 1
            orientation = 1 if gender == 0 and np.random.random() < queer_ratio else 0
            age = np.random.randint(15,65)
            if age >= 15 and age < 35:
                sexual_activity = 1 if np.random.random() < self.sexual_activity_high else 0
            else:
                sexual_activity = 0
            
            p = person(i,gender, orientation, age, sexual_activity)
            self.persons.append(p)
            if gender == 0 and orientation == 0:
                p.sec_id = len(self.straight_males)
                self.straight_males.append(p)
            elif gender == 0 and orientation == 1:
                p.sec_id = len(self.homosexual_males)
                self.homosexual_males.append(p)
            else:
                p.sec_id = len(self.females)
                self.females.append(p)
        self.number_of_singles = number_of_persons
    
    def replace_person(self,p):
        identifier = p.identifier
        sec_id = p.sec_id
        gender = p.gender
        orientation = p.orientation
        age = 15
        sexual_activity = 1 if np.random.random() < self.sexual_activity_high else 0
    
        #remove p from current partners from all p's current partners
        for j in p.current_partners:
            o = self.persons[j]
            o.remove_partner(p,self)
        
        #update system numbers
        if p.number_of_partners > 0:
            self.number_of_singles += 1
        p.cure(self)
        
        #remove partnerships where p is involved in
        for i,ps in enumerate(self.partnerships):
            if p.identifier in [ps.persons[0].identifier,ps.persons[1].identifier]:
                #deleting steady partnership
                if ps.type == 1:
                    self.number_of_steady -= 1
                self.partnerships[i] = None
                
        self.partnerships = list(filter(None,self.partnerships))
            
        new_person = person(identifier,gender, orientation, age, sexual_activity)
        new_person.sec_id = sec_id
        
        #update lists of persons
        self.persons[identifier] = new_person
        if gender == 0 and orientation == 0:
            self.straight_males[sec_id] = new_person
        elif gender == 0 and orientation == 1:
            self.homosexual_males[sec_id] = new_person
        elif gender == 1:
            self.females[sec_id] = new_person
    
    def print_state(self):
        print("id\tgender\torientation\tage\tactivity\tdisease\ttime since inf\t#partners\tcurrent partners")
        for p in self.persons:
            p.print_state()
        print("type\tpersons")
        for ps in self.partnerships:
            ps.print_state()
            
    def time_step(self):
        n = int(len(self.persons)/2) - len(self.partnerships)
        
        #creation of partnerships
        for i in range(n):
            if np.random.random() < self.rho:
                formed = False
                partnership_type = 1 if np.random.random() < self.f else 2
                
                #add a end condition
                iteration = 0
                while(not formed and iteration < len(self.persons)):
                    #form partnership
                    y = np.random.choice(self.straight_males)
                    x = np.random.choice(self.females)
                    
                    exists = False
                    mixing_prob = mixing_probability(x,y,partnership_type)
                    if np.random.random() < mixing_prob:
                        for ps in self.partnerships:
                            if y in ps.persons and x in ps.persons:
                                exists = True
                        if not exists:
                            p = Partnership(partnership_type,x,y)
                            x.add_partner(y,self)
                            y.add_partner(x,self)
                            self.partnerships.append(p)
                            formed = True
                            
                            if partnership_type == 1:
                                self.number_of_steady += 1
                    iteration += 1
        
        #Disease transmission
        for i,partnership in enumerate(self.partnerships):
            p1, p2 = partnership.persons
            s1 = p1.disease_status
            s2 = p2.disease_status
            partnership_type = partnership.type
            
            if s1 > 0 and s2 == 0:
                infected = p1
                non_infected = p2
            elif s1 == 0 and s2 > 0:
                infected = p2
                non_infected = p1
            else:
                continue
            
            #extend with MSM
            
            if infected.gender == 0 and non_infected.gender == 1:
                #male -> female
                if partnership_type == 1:
                    if np.random.random() < self.transmission_male_female_steady:
                        if np.random.random() < self.asymptomatic_women:
                            non_infected.asymptomatic_infection(self)
                        else:
                            non_infected.symptomatic_infection(self)
                elif partnership_type == 2:
                    if np.random.random() < self.transmission_male_female_casual:
                        if np.random.random() < self.asymptomatic_women:
                            non_infected.asymptomatic_infection(self)
                        else:
                            non_infected.symptomatic_infection(self)
            elif infected.gender == 1 and non_infected.gender == 0:
                #female -> male
                if partnership_type == 1:
                    if np.random.random() < self.transmission_female_male_steady:
                        if np.random.random() < self.asymptomatic_men:
                            non_infected.asymptomatic_infection(self)
                        else:
                            non_infected.symptomatic_infection(self)
                elif partnership_type == 2:
                    if np.random.random() < self.transmission_female_male_casual:
                        if np.random.random() < self.asymptomatic_men:
                            non_infected.asymptomatic_infection(self)
                        else:
                            non_infected.symptomatic_infection(self)
        
        #Separation of partnerships
        for i,ps in enumerate(self.partnerships):
            partnership_type = ps.type
            p1, p2 = ps.persons
            
            #check partnership type
            if partnership_type == 1:
                if np.random.random() < self.sigma1:
                    p1.remove_partner(p2,self)
                    p2.remove_partner(p1,self)
                    self.partnerships[i] = None
                    self.number_of_steady -= 1
            elif partnership_type == 2:
                if np.random.random() < self.sigma2:
                    p1.remove_partner(p2,self)
                    p2.remove_partner(p1,self)
                    self.partnerships[i] = None
        self.partnerships = list(filter(None,self.partnerships))
        
        #Replacement
        for p in self.persons:
            if p.age == 65:
                #replace with new person
                self.replace_person(p)
        
        #Recovery
        for p in self.persons:
            if p.gender == 0 and p.disease_status == 1:
                if np.random.random() < self.recovery_rate_symptomatic_men:
                    p.cure(self)
            elif p.gender == 0 and p.disease_status == 2:
                if np.random.random() < self.recovery_rate_asymptomatic_men:
                    p.cure(self)
            elif p.gender == 1 and p.disease_status == 1:
                if np.random.random() < self.recovery_rate_symptomatic_women:
                    p.cure(self)
            elif p.gender == 1 and p.disease_status == 2:
                if np.random.random() < self.recovery_rate_asymptomatic_women:
                    p.cure(self)
                
        #Treatment
        for p in self.persons:
            if p.disease_status == 1:
                if p.gender == 0:
                    if p.time_since_infection >= self.incubation_time_men + self.patient_delay_treatment_men:
                        p.cure(self)
                elif p.gender == 1:
                    if p.time_since_infection >= self.incubation_time_women + self.patient_delay_treatment_women:
                        p.cure(self)
#                 #contact tracing
#                 for j in p.current_partners:
#                     o = self.persons[j]
#                     o.cure(self)
        
#         #Screening
#         if self.time % 365 == 0:
#             number = int(len(self.persons)*self.screening_percentage)
#             screened = np.random.choice(self.persons,number)
#             for p in screened:
#                 p.cure(self)
        
        #Increase time
        self.time += 1
        for p in self.persons:
            if p.disease_status > 0:
                p.time_since_infection += 1
            if p.gender == 0 and p.time_since_infection > self.incubation_time_men:
                p.symptomatic_infection(self)
            elif p.gender == 1 and p.time_since_infection > self.incubation_time_women:
                p.symptomatic_infection(self)
        
        if self.time % 365 == 0:
            for i,p in enumerate(self.persons):
                p.age += 1
        
def mixing_probability(x,y,partnership_type):
    j = age_group(x)
    k = age_group(y)
    
    if j == k:
        phi_a = 1
    else:
        phi_a = 0.2**(np.abs(j + 1 - k))
    
    # c = sexual_activity
    # d = number_of_partners
    
    if partnership_type == 1:
        if x.number_of_partners == 0 and y.number_of_partners == 0:
            phi_cd = 1
        else:
            phi_cd = 0
    else:
        if x.sexual_activity == 1 and y.sexual_activity == 1:
            phi_cd = 1
        elif x.sexual_activity == 1 and y.sexual_activity == 0\
        and y.number_of_partners == 0:
            phi_cd = 0.1
        elif y.sexual_activity == 1 and x.sexual_activity == 0\
        and x.number_of_partners == 0:
            phi_cd = 0.1
        elif x.sexual_activity == 0 and x.number_of_partners == 0\
        and y.sexual_activity == 0 and y.number_of_partners == 0:
            phi_cd = 0.01
        else:
            phi_cd = 0
    return phi_a * phi_cd
        
def age_group(p):
    a = p.age
    if a >= 15 and a < 25:
        age_group = 1
    elif a >= 25 and a < 35:
        age_group = 2
    elif a >= 35 and a < 45:
        age_group = 3
    elif a >= 45 and a < 55:
        age_group = 4
    else:
        age_group = 5
    return age_group         

In [ ]:
s = system()
s.set_constants(sexual_activity_high=1)
s.initialize(250,0.5,0)

for i in range(2000):
    s.time_step()
for i in range(150):
    p = np.random.choice(s.persons)
    p.asymptomatic_infection(s)
for i in range(25):
    p = np.random.choice(s.persons)
    p.symptomatic_infection(s)
s.time = 1


In [ ]:
s.print_state()

In [ ]:
import time
singles_list = []
steady_list = []
partnerships_list = []
symptomatic_list = []
asymptomatic_list = []
start = time.time()
for i in range(2500):
    s.time_step()
    singles_list.append(s.number_of_singles)
    steady_list.append(s.number_of_steady)
    partnerships_list.append(len(s.partnerships))
    symptomatic_list.append(s.number_of_symptomatic)
    asymptomatic_list.append(s.number_of_asymptomatic)
print(time.time() - start)

In [ ]:
s.print_state()

In [ ]:
fig, ax = plt.subplots(figsize=(15,10))
ax.plot(singles_list,label="number of singles")
ax.plot(steady_list,label="number of steady partnerships")
ax.plot(partnerships_list,label="number of partnerships")
ax.plot(len(s.persons)-np.array(singles_list),label="number of persons in a partnership")
ax.plot(np.array(partnerships_list) - np.array(steady_list),label="number of casual partnerships")
ax.plot(symptomatic_list,label="symptomatically infected")
ax.plot(asymptomatic_list,label="asymptomatically infected")
ax.legend()

In [ ]:
fig,ax = plt.subplots(figsize=(15,10))
ax.plot(symptomatic_list,label="symptomatically infected")
ax.plot(asymptomatic_list,label="asymptomatically infected")
ax.legend()

In [ ]:
G = nx.Graph()
G.add_nodes_from([x for x in range(len(s.persons))])
edges_list = []
edge_type =[]
for ps in s.partnerships:
    p1, p2 = ps.persons
    edges_list.append((p1.identifier,p2.identifier))
    edge_type.append(ps.type)
G.add_edges_from(edges_list)
    
infection_status = []
for p in s.persons:
    infection_status.append(p.disease_status)
plt.figure(figsize=(15,15))
nx.draw(G,node_size=8,node_color=infection_status,edge_color=edge_type,with_labels=True,font_size=8)
plt.savefig("test.pdf")

In [ ]:
s.print_state()